# Export du modèle YOLO11n de base (non fine-tuné)
- Modèle pré-entraîné sur COCO (80 classes)
- Test sur COCO128 (sous-ensemble de 128 images), puis export en ONNX et blob

In [14]:
import os
from pathlib import Path

import blobconverter
from ultralytics import YOLO

ROOT_DIR = Path(__file__).parent.parent.resolve() if "__file__" in dir() else Path.cwd().parent.resolve()

In [15]:
BASE_WEIGHTS = str(ROOT_DIR / "models/base/yolo11n.pt")
OUTPUT_DIR = str(ROOT_DIR / "models/base")

IMGSZ = 640
CONF = 0.25

In [16]:
model = YOLO(BASE_WEIGHTS)
print(f"Modele charge: {BASE_WEIGHTS}")
print(f"Nombre de classes: {len(model.names)}")
print(f"Classes: {list(model.names.values())[:10]}...")  # Affiche les 10 premieres

Modele charge: /home/nathan/Documents/GitHub/3d_edge_object_detection/models/base/yolo11n.pt
Nombre de classes: 80
Classes: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light']...


In [17]:
# Validation sur COCO128 (telecharge automatiquement ~7MB)
metrics = model.val(data="coco128.yaml", imgsz=IMGSZ, device=0)

print("\n" + "="*50)
print("Resultats sur COCO128")
print("="*50)
print(f"mAP50-95: {float(metrics.box.map):.4f}")
print(f"mAP50   : {float(metrics.box.map50):.4f}")
print(f"mAP75   : {float(metrics.box.map75):.4f}")
print(f"Precision: {float(metrics.box.mp):.4f}")
print(f"Recall   : {float(metrics.box.mr):.4f}")

Ultralytics 8.3.241 🚀 Python-3.11.14 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7806MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

WARNING ⚠️ Dataset 'coco128.yaml' images not found, missing path '/home/nathan/Documents/education/master_2/cours/deep_learning_for_image/TP6/datasets/coco128/images/train2017'
Unzipping /home/nathan/Documents/education/master_2/cours/deep_learning_for_image/TP6/datasets/coco128.zip to /home/nathan/Documents/education/master_2/cours/deep_learning_for_image/TP6/datasets/coco128...: 100% ━━━━━━━━━━━━ 263/263 3.8Kfiles/s 0.1s
Dataset download success ✅ (1.3s), saved to /home/nathan/Documents/education/master_2/cours/deep_learning_for_image/TP6/datasets

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1244.7±440.5 MB/s, size: 47.0 KB)
val: Scanning /home/nathan/Documents/education/master_2/cours/deep_learning_for_image/TP6/datasets/coco128/labels/train2017... 126 images, 2 backgrounds, 0 corrupt: 10

In [11]:
onnx_path = model.export(
    format="onnx",
    imgsz=IMGSZ,
    opset=13,
    simplify=True,
    dynamic=False,
)

print("ONNX exporte:", onnx_path)

Ultralytics 8.3.241 🚀 Python-3.11.14 torch-2.9.1+cu128 CPU (Intel Core i9-14900HX)

PyTorch: starting from '/home/nathan/Documents/GitHub/3d_edge_object_detection/models/base/yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (5.4 MB)

ONNX: starting export with onnx 1.20.0 opset 13...
ONNX: slimming with onnxslim 0.1.80...
ONNX: export success ✅ 0.6s, saved as '/home/nathan/Documents/GitHub/3d_edge_object_detection/models/base/yolo11n.onnx' (10.2 MB)

Export complete (0.7s)
Results saved to /home/nathan/Documents/GitHub/3d_edge_object_detection/models/base
Predict:         yolo predict task=detect model=/home/nathan/Documents/GitHub/3d_edge_object_detection/models/base/yolo11n.onnx imgsz=640  
Validate:        yolo val task=detect model=/home/nathan/Documents/GitHub/3d_edge_object_detection/models/base/yolo11n.onnx imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app
ONNX exporte: /home/nathan/Do

In [12]:
blob_path = blobconverter.from_onnx(
    model=onnx_path,
    data_type="FP16",
    shaves=6,
    output_dir=OUTPUT_DIR,
)

print("BLOB exporte:", blob_path)

[==================================================]
Done
BLOB exporte: /home/nathan/Documents/GitHub/3d_edge_object_detection/models/base/yolo11n_openvino_2022.1_6shave.blob


In [13]:
print("\n" + "="*50)
print("RESUME - Modele YOLO11n BASE (non fine-tune)")
print("="*50)
print(f"Classes: 80 (COCO)")
print(f"Input size: {IMGSZ}x{IMGSZ}")
print(f"Fichiers dans: {OUTPUT_DIR}")
print(f"  - PT:   {BASE_WEIGHTS}")
print(f"  - ONNX: {onnx_path}")
print(f"  - BLOB: {blob_path}")


RESUME - Modele YOLO11n BASE (non fine-tune)
Classes: 80 (COCO)
Input size: 640x640
Fichiers dans: /home/nathan/Documents/GitHub/3d_edge_object_detection/models/base
  - PT:   /home/nathan/Documents/GitHub/3d_edge_object_detection/models/base/yolo11n.pt
  - ONNX: /home/nathan/Documents/GitHub/3d_edge_object_detection/models/base/yolo11n.onnx
  - BLOB: /home/nathan/Documents/GitHub/3d_edge_object_detection/models/base/yolo11n_openvino_2022.1_6shave.blob
